In [7]:
# sobre el dataset de la competición de kaggle de diamantes de méxico

# imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# read CSV
df = pd.read_csv('data/diamonds_train.csv')
df.head(10)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,1.21,Premium,J,VS2,62.4,58.0,4268,6.83,6.79,4.25
1,0.32,Very Good,H,VS2,63.0,57.0,505,4.35,4.38,2.75
2,0.71,Fair,G,VS1,65.5,55.0,2686,5.62,5.53,3.65
3,0.41,Good,D,SI1,63.8,56.0,738,4.68,4.72,3.00
4,1.02,Ideal,G,SI1,60.5,59.0,4882,6.55,6.51,3.95
5,1.52,Ideal,F,SI2,61.2,57.0,9057,7.45,7.39,4.54
6,1.01,Fair,H,VS2,65.2,56.0,3733,6.23,6.19,4.05
7,1.52,Ideal,J,VS1,62.3,58.0,8608,7.32,7.35,4.57
8,0.31,Ideal,H,VS1,61.8,54.2,557,4.33,4.37,2.69
9,1.14,Ideal,G,SI1,61.7,56.0,6741,6.71,6.75,4.15


In [3]:
# Analize

df.dtypes

carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
price        int64
x          float64
y          float64
z          float64
dtype: object

In [10]:
# Transformation: Label encoding

from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

for col in ('cut', 'color'): 
    # se le debe pasar un dict al labelencoder
    df[col] = labelencoder.fit_transform(df[col])   

df.head(10)

,carat,cut,color,depth,table,price,x,y,z,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1.21,3,6,62.4,58.0,4268,6.83,6.79,4.25,0,0,0,0,1,0,0
1,0.32,4,4,63.0,57.0,505,4.35,4.38,2.75,0,0,0,0,1,0,0
2,0.71,0,3,65.5,55.0,2686,5.62,5.53,3.65,0,0,0,1,0,0,0
3,0.41,1,0,63.8,56.0,738,4.68,4.72,3.00,0,1,0,0,0,0,0
4,1.02,2,3,60.5,59.0,4882,6.55,6.51,3.95,0,1,0,0,0,0,0
5,1.52,2,2,61.2,57.0,9057,7.45,7.39,4.54,0,0,1,0,0,0,0
6,1.01,0,4,65.2,56.0,3733,6.23,6.19,4.05,0,0,0,0,1,0,0
7,1.52,2,6,62.3,58.0,8608,7.32,7.35,4.57,0,0,0,1,0,0,0
8,0.31,2,4,61.8,54.2,557,4.33,4.37,2.69,0,0,0,1,0,0,0
9,1.14,2,3,61.7,56.0,6741,6.71,6.75,4.15,0,1,0,0,0,0,0


In [8]:
# Transformation: One hot encoding

# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder(handle_unknown='ignore')

df = pd.get_dummies(df, prefix='clarity', columns=['clarity'], drop_first=True)  
df.head()

,carat,cut,color,depth,table,price,x,y,z,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1.21,Premium,J,62.4,58.0,4268,6.83,6.79,4.25,0,0,0,0,1,0,0
1,0.32,Very Good,H,63.0,57.0,505,4.35,4.38,2.75,0,0,0,0,1,0,0
2,0.71,Fair,G,65.5,55.0,2686,5.62,5.53,3.65,0,0,0,1,0,0,0
3,0.41,Good,D,63.8,56.0,738,4.68,4.72,3.00,0,1,0,0,0,0,0
4,1.02,Ideal,G,60.5,59.0,4882,6.55,6.51,3.95,0,1,0,0,0,0,0


In [11]:
df.dtypes

carat           float64
cut               int64
color             int64
depth           float64
table           float64
price             int64
x               float64
y               float64
z               float64
clarity_IF        uint8
clarity_SI1       uint8
clarity_SI2       uint8
clarity_VS1       uint8
clarity_VS2       uint8
clarity_VVS1      uint8
clarity_VVS2      uint8
dtype: object

In [12]:
target_col = 'price'
X = df[[col for col in df.columns if col != target_col]]
y = df[target_col]


In [13]:
# split: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

SIZE_PERC = 0.2
STATE = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=SIZE_PERC, random_state=STATE)

In [ ]:
regr = RandomForestRegressor()
param_grid = {
    'n_estimators': [400,500,600],
    'min_samples_split': [2],
    'min_samples_leaf': [0.5, 1, 0.3],
    'max_depth': [None]
}

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator = regr, param_grid = param_grid, cv = 2, verbose = 2)
grid_search.fit(X_train, y_train)
grid_search.best_estimator_.fit(X_train, y_train)
y_pred = grid_search.best_estimator_.predict(X_test)
mean_squared_error(y_test, y_pred)**0.5
r2_score(y_test, y_pred)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV] max_depth=None, min_samples_leaf=0.5, min_samples_split=2, n_estimators=400 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=None, min_samples_leaf=0.5, min_samples_split=2, n_estimators=400, total=   1.0s
[CV] max_depth=None, min_samples_leaf=0.5, min_samples_split=2, n_estimators=400 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV]  max_depth=None, min_samples_leaf=0.5, min_samples_split=2, n_estimators=400, total=   0.9s
[CV] max_depth=None, min_samples_leaf=0.5, min_samples_split=2, n_estimators=500 
[CV]  max_depth=None, min_samples_leaf=0.5, min_samples_split=2, n_estimators=500, total=   1.2s
[CV] max_depth=None, min_samples_leaf=0.5, min_samples_split=2, n_estimators=500 
[CV]  max_depth=None, min_samples_leaf=0.5, min_samples_split=2, n_estimators=500, total=   1.1s
[CV] max_depth=None, min_samples_leaf=0.5, min_samples_split=2, n_estimators=600 
[CV]  max_depth=None, min_samples_leaf=0.5, min_samples_split=2, n_estimators=600, total=   1.4s
[CV] max_depth=None, min_samples_leaf=0.5, min_samples_split=2, n_estimators=600 
[CV]  max_depth=None, min_samples_leaf=0.5, min_samples_split=2, n_estimators=600, total=   1.5s
[CV] max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=400 
[CV]  max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=400, total=  47.9s
[CV] max_dep